<a href="https://colab.research.google.com/github/IseLei18-code/NBA-picks/blob/main/Sihaya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Introduction**

This following data analysis project aims to use past and recent data from NBA players and determine not only the bets on prizebets, more or less, and also the probability of us winning.

## **Small Simulator**
Demonstration of the model

In [1]:
%pip install -q nba_api
import pandas as pd
pd.set_option('display.max_columns', None)
import re
import numpy as np
import time
import requests
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import LeagueDashPlayerStats

player_name = input("Enter Player: ")
stats = input("Stats:")

ldr = LeagueDashPlayerStats(
    season='2025-26',
    season_type_all_star='Regular Season',
    measure_type_detailed_defense='Base',
    per_mode_detailed='PerMinute',
    last_n_games=10
)
df = ldr.get_data_frames()[0]
df.head()

row_index = df['PLAYER_NAME'].equals(player_name)
print(row_index)
df.iloc[row_index, 0]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 4.6 MB/s eta 0:00:00
Enter Player: Stephen Curry
Stats:pts
False


ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

# EDA And Data Cleaning


In [ ]:
%pip install -q nba_api
import pandas as pd
pd.set_option('display.max_columns', None)
import re
import numpy as np
import time
import requests
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import LeagueDashPlayerStats

In [ ]:
test_url = 'https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=PerGame&Scope=S&Season=2025-26&SeasonType=Regular%20Season&StatCategory=PTS'
test_perMin_last = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=10&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerMinute&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2025-26&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

In [15]:
# manipulate url to get datas from NBA Stats Leaders

ldr = LeagueDashPlayerStats(
    season='2025-26',
    season_type_all_star='Regular Season',
    measure_type_detailed_defense='Base',
    per_mode_detailed='PerGame',
    last_n_games=10
)
df = ldr.get_data_frames()[0]
df.head()

curry_rows = df[df['PLAYER_NAME'] == 'Stephen Curry']
curry_rows
print(curry_rows)


     PLAYER_ID    PLAYER_NAME NICKNAME     TEAM_ID TEAM_ABBREVIATION   AGE  \
404     201939  Stephen Curry  Stephen  1610612744               GSW  37.0   

     GP  W  L  W_PCT   MIN  FGM   FGA  FG_PCT  FG3M  FG3A  FG3_PCT  FTM  FTA  \
404   7  4  3  0.571  29.5  7.7  18.4   0.419   3.6  10.9    0.329  5.4  6.0   

     FT_PCT  OREB  DREB  REB  AST  TOV  STL  BLK  BLKA   PF  PFD   PTS  \
404   0.905   0.1   3.0  3.1  3.6  2.4  1.1  0.1   0.4  2.4  4.9  24.4   

     PLUS_MINUS  NBA_FANTASY_PTS  DD2  TD3  WNBA_FANTASY_PTS  GP_RANK  W_RANK  \
404        -0.7             35.0    0    0              37.3      268     193   

     L_RANK  W_PCT_RANK  MIN_RANK  FGM_RANK  FGA_RANK  FG_PCT_RANK  FG3M_RANK  \
404     238         203        96        29        15          291          8   

     FG3A_RANK  FG3_PCT_RANK  FTM_RANK  FTA_RANK  FT_PCT_RANK  OREB_RANK  \
404          2           214        22        30           75        387   

     DREB_RANK  REB_RANK  AST_RANK  TOV_RANK  STL_RANK

In [ ]:
# print columns:
stats_col = list(r['resultSet']['headers'])
print(stats_col)

['PLAYER_ID', 'RANK', 'PLAYER', 'TEAM_ID', 'TEAM', 'GP', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PTS', 'EFF']


In [ ]:
pd.DataFrame(r['resultSet']['rowSet'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,203507,1,Giannis Antetokounmpo,1610612749,MIL,9,32.3,13.1,20.8,0.631,0.8,1.6,0.500,6.8,10.7,0.635,4.0,8.3,12.3,6.2,0.9,1.1,2.8,33.8,40.0
1,1630178,2,Tyrese Maxey,1610612755,PHI,10,41.1,10.9,23.2,0.470,4.1,9.3,0.441,7.3,8.4,0.869,0.4,4.5,4.9,8.2,1.2,1.0,2.7,33.2,32.4
2,1628983,3,Shai Gilgeous-Alexander,1610612760,OKC,11,34.8,11.3,21.6,0.521,2.1,6.0,0.348,8.5,9.5,0.904,0.4,4.8,5.2,6.0,1.1,1.1,1.7,33.2,33.5
3,1628378,4,Donovan Mitchell,1610612739,CLE,9,33.1,10.4,19.6,0.534,4.3,9.8,0.443,5.4,6.4,0.845,0.2,3.0,3.2,5.1,1.6,0.4,3.1,30.7,27.8
4,1630559,5,Austin Reaves,1610612747,LAL,8,36.8,9.1,19.0,0.480,2.9,8.9,0.324,9.1,10.4,0.880,0.5,4.6,5.1,9.0,1.5,0.0,3.1,30.3,31.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,1631172,281,Ousmane Dieng,1610612760,OKC,8,10.1,0.5,2.4,0.211,0.3,1.4,0.182,0.3,0.3,1.000,0.3,1.8,2.0,0.5,0.1,0.6,0.3,1.5,2.6
283,1641763,284,Julian Phillips,1610612741,CHI,8,5.9,0.5,1.8,0.286,0.1,0.8,0.167,0.0,0.0,0.000,0.5,0.1,0.6,0.1,0.4,0.3,0.0,1.1,1.3
284,1642358,285,AJ Johnson,1610612764,WAS,7,5.0,0.3,1.4,0.200,0.0,0.3,0.000,0.4,0.6,0.750,0.3,0.3,0.6,0.6,0.0,0.0,0.1,1.0,0.7
285,1642964,286,Brooks Barnhizer,1610612760,OKC,9,6.7,0.4,1.0,0.444,0.0,0.1,0.000,0.0,0.0,0.000,0.3,1.3,1.7,0.2,0.1,0.1,0.1,0.9,2.3


In [ ]:
# test last 3, 5, 10 pts for curry
test_perMin_last = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=10&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerMinute&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2025-26&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
r = requests.get(url=test_perMin_last).json()
df = pd.DataFrame(r['resultSet']['rowSet'])
df



KeyboardInterrupt: 

# NLP (Regex for now)
Take in user query, parse, and extract key terms using Regex.

Sets variables:
*   player_name (Lebron James)
*   stats (pts, ast, reb)
*   over (True) and under (False)
*   value (25)

User query: Stephen Curry over 25 points
Variables:
*   player_name = 'Stephen Curry'
*   stats = 'pts'
*   over = True
*   value = 25


In [18]:
STAT_MAP = {
    "pt": "pts", "pts": "pts", "point": "pts", "points": "pts",
    "reb": "reb", "rebs": "reb", "rebound": "reb", "rebounds": "reb",
    "ast": "ast", "asts": "ast", "assist": "ast", "assists": "ast",
    "blk": "blk", "blks": "blk", "block": "blk", "blocks": "blk",
    "stl": "stl", "stls": "stl", "steal": "stl", "steals": "stl"
}

pattern = re.compile(
    r"""
    ^\s*
    (?P<player>[\w\.\-\'\s]+?)      # player name
    \s+
    (?P<dir>over|under)             # over / under
    \s+
    (?P<value>\d+(?:\.\d+)?)        # number like 25 or 8.5
    \s+
    (?P<stat>
        points?|pts?|pt|           # points
        rebounds?|rebs?|reb|       # rebounds
        assists?|asts?|ast|        # assists
        blocks?|blks?|blk|         # blocks
        steals?|stls?|stl          # steals
    )
    \s*$
    """,
    re.IGNORECASE | re.VERBOSE,
)

def parse_user_query(query: str) -> dict:
    m = pattern.match(query)
    if not m:
        raise ValueError(f"Could not parse query: {query}")

    player_name = m.group("player").strip()
    direction = m.group("dir").lower()
    over = direction == "over"
    raw_value = m.group("value")
    value = float(raw_value) if "." in raw_value else int(raw_value)

    stat_word = m.group("stat").lower()
    # normalize to pts / ast / reb
    key = next(k for k in STAT_MAP if stat_word.startswith(k))
    stats = STAT_MAP[key]

    return {
        "player_name": player_name,
        "stats": stats,
        "over": over,
        "value": value,
    }

# Example:
user_query = input("Enter Query:")
vars_parsed = parse_user_query(user_query)
print(vars_parsed)

Enter Query:Stephen Curry over 25 points
{'player_name': 'Stephen Curry', 'stats': 'pts', 'over': True, 'value': 25}


# Models
3 models will be used

## Model 1 (Elastic Net Linear Regression):
Focuses on the player's most recent games played (3, 5, 7, 10) and considers the other metrics that may affect the player's behavior.

Retrieve data from https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=10&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerMinute&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2025-26&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight=

Change LastNGames=3, 5, 7, 10
Change PerMode = PerMinute